# Spatial aggregation by level

This notebook allows to compile the results of the person exposed building exposed and cost platform on a larger scale (nuts2 nuts1)

## gives the results on a larger scale

### install the libraries

In [ ]:
import pandas as pd
import os
from urllib.parse import urlparse
import requests

What information are you looking for?

In [ ]:
value_type = "number"# The possible values are population, number or cost
nuts_value = "DEF0"
year = "2100"# The possible values are 2010 , 2030, 2050, 2100
rp="100"# The possible values are static, 1, 100 or 1000
defense="HIGH_DEFENDED" # The possible values are HIGH_DEFENDED, LOW_DEFENDED or UNDEFENDED
ssp="SSP126" # The possible values are SSP126, SSP245 or SSP585

connection to the catalog and download of the file

In [ ]:

defense=defense+"_MAPS"

if value_type == "population":
    path="https://storage.googleapis.com/coclico-data-public/coclico/pp_stats/map_stats/"+defense+"/"+rp+"/"+ssp+"/pop_stats_"+defense+"_"+rp+"_"+ssp+"_"+year+".parquet"
    local_path = "CoCliCo_data/people.parquet"
    response = requests.get(path)
    with open(local_path, "wb") as f:
        f.write(response.content)
    colum="abs_affected"
elif value_type == "number":
    path="https://storage.googleapis.com/coclico-data-public/coclico/be_stats/map_stats/"+defense+"/"+rp+"/"+ssp+"/be_stats_"+defense+"_"+rp+"_"+ssp+"_"+year+".parquet"
    local_path = "CoCliCo_data/number.parquet"
    response = requests.get(path)
    with open(local_path, "wb") as f:
        f.write(response.content)  
    colum="abs_affected"       
elif value_type == "cost":
    path = "https://storage.googleapis.com/coclico-data-public/coclico/bc_stats/bc_stats.parquet"
    local_path = "CoCliCo_data/cost.parquet"
    response = requests.get(path)
    with open(local_path, "wb") as f:
        f.write(response.content)
    colum=defense+"\\"+rp+"\\"+ssp+"\\"+year+"\\buildings"


else:
    raise ValueError(f"Unknown value_type '{value_type}'")


defines what scale is requested

In [ ]:
def detect_scale(code, df):
    """
    Detects the scale  of the given code.
    """
    if code in df['CNTR_CODE'].values:
        return 'CNTR_CODE'
    elif code in df['nuts_2'].values:
        return 'nuts_2'
    elif code in df['nuts_3'].values:
        return 'nuts_3'
    else:
        return 'unknown'


In [ ]:
df=pd.read_parquet(local_path)
scale=detect_scale(nuts_value,df)
print(scale)
filtered = df[df[scale] == nuts_value]
summary = filtered.groupby(scale)[colum].sum().reset_index()
print(summary)

nuts_2
  nuts_2  abs_affected
0   DEF0       36120.0
